In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
import bs4 as bs
import urllib.request # import libraries for data processing
import requests

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M' # i
page = urllib.request.urlopen(url).read()  
soup = bs.BeautifulSoup(page,'lxml') # initializing beautifulsoup

table = soup.find('table', attrs={'class':"wikitable"}) # search the page html and find the table tag and extract the wikitable class for further processing
table_rows = table.find_all('tr') # find all the tr tags in the table and put in into a variable for further processing

C = []   # Crtea a empty list for adding results in
for tr in table_rows:
    td = tr.find_all('td')   # loop through table_rows and search for td tags 
    row = [tr.text for tr in td] 
    C.append(row) # append the td data in the empty list with data that was found in table rows

In [3]:
df = pd.DataFrame(C, columns=["Postal Code", "Borough", "Neighborhood"]).drop([0], axis=0) # Placed the scraped data into a dataframe with 3 columns and dropped the first row since it was populated with none values, axis = 0 drops rows 
df.replace(r'\n','', regex=True, inplace=True) # Replacing the \n with '', after I creted the dataframe I noticed the newline character also imported so I had to strip it from the columns

In [4]:
indexNames = df[df['Borough']!="Not assigned"] # extract data from dataframe and only return where values are not equal to Not assigned
indexNames.replace(r' / ',', ', regex=True, inplace=True) # Replacing the / with ,
indexNames.reset_index(drop=True, inplace=True) # reseting the indexes since the returned dataframe indexes did not start at 0 but returned the indexes that was not dropped
indexNames.head()

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4263: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [5]:
df_ll = pd.read_csv('Geospatial_Coordinates.csv')
df_ll.head()

df2 = pd.merge(indexNames,
                 df_ll[['Postal Code','Latitude', 'Longitude']],
                 on='Postal Code')
df2.shape

(103, 5)

In [6]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="Toronto")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('Coordinates of Toronto are {}, {}.'.format(latitude, longitude))

Coordinates of Toronto are 43.6534817, -79.3839347.


In [7]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, Neighbourhood in zip(df2['Latitude'], df2['Longitude'], df2['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(Neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [8]:
CLIENT_ID = 'VFRXIQBZ3D34PYY0EFFAVEA2DRMQQEDUAT0PXNPEG4HMIUP1' # your Foursquare ID
CLIENT_SECRET = 'QVW4ZFONYCHGWRU5H4SWGYXRQF0T3J0DSTUR1XPLEHSOFSIB' # your Foursquare Secret
VERSION = '20200401' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: VFRXIQBZ3D34PYY0EFFAVEA2DRMQQEDUAT0PXNPEG4HMIUP1
CLIENT_SECRET:QVW4ZFONYCHGWRU5H4SWGYXRQF0T3J0DSTUR1XPLEHSOFSIB


In [9]:
# defining radius and limit of venues to get
radius=500
LIMIT=100

In [10]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [11]:
venues = getNearbyVenues(names=df2['Neighborhood'],
                                   latitudes=df2['Latitude'],
                                   longitudes=df2['Longitude']
                                  ).head(30)

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmount Park
Bayview Village
Downsview
The Danforth West, Ri

In [12]:
venues.shape

(30, 7)